In [1]:
import pygame
import numpy as np

# --- Pygame 초기화 ---
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D EKF 시뮬레이션 (원형 궤도 + 속도 보상)") # 창 제목 변경
clock = pygame.time.Clock()

# --- 시뮬레이션 파라미터 ---
dt = 1.0
accel_noise_std = 0.02 # 제어 효과를 보기 위해 약간 줄임
measurement_noise_std = 5.0 # 위치 측정 잡음 (이 코드에서는 사용 안함)
num_steps = 600
K_DRAG = 0.001  # 공기 저항 계수 (적절히 조절)

# --- 원형 궤도 파라미터 ---
ORBIT_CENTER_X = WIDTH / 2.0
ORBIT_CENTER_Y = HEIGHT / 2.0
ORBIT_RADIUS = min(WIDTH, HEIGHT) / 3.0
TARGET_ANGULAR_VELOCITY = 0.04 # 라디안/스텝
V_TARGET = TARGET_ANGULAR_VELOCITY * ORBIT_RADIUS # 목표 접선 속도
ACCEL_C_MAG_CMD = TARGET_ANGULAR_VELOCITY**2 * ORBIT_RADIUS # 기본 구심 가속도 크기


# --- 속도 제어 파라미터 ---
KP_SPEED = 0.1  # 속도 오차에 대한 비례 게인 (튜닝 필요)

# --- 시스템 상태 정의 ---
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]]) # 측정 행렬 (위치 x,y 직접 측정)

Q_diag = np.array([
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2
])
Q = np.diag(Q_diag)
R = np.eye(2) * measurement_noise_std**2 # 위치 측정 잡음 공분산

# --- 초기 실제 상태 (원형 궤도 시작점) ---
initial_px = ORBIT_CENTER_X + ORBIT_RADIUS
initial_py = ORBIT_CENTER_Y
initial_vx = 0.0
initial_vy = V_TARGET # 목표 접선 속도로 시작
initial_true_state = np.array([initial_px, initial_py, initial_vx, initial_vy])

# --- EKF 초기 추정값 ---
x_est_initial = np.array([initial_px + 15, initial_py - 15, initial_vx, initial_vy - 0.5])

# --- 초기 추정 공분산 ---
P_initial = np.eye(4) * 50.0

# --- EKF 관련 함수 정의 (이전과 동일) ---
def state_transition_function_f(x_prev, u_cmd, dt_val, k_d_val):
    px, py, vx, vy = x_prev
    ax_cmd, ay_cmd = u_cmd
    speed = np.sqrt(vx**2 + vy**2)
    if speed < 1e-6:
        ax_drag = 0
        ay_drag = 0
    else:
        ax_drag = -k_d_val * vx * speed
        ay_drag = -k_d_val * vy * speed
    ax_net = ax_cmd + ax_drag
    ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt_val + 0.5 * ax_net * dt_val**2
    py_next = py + vy * dt_val + 0.5 * ay_net * dt_val**2
    vx_next = vx + ax_net * dt_val
    vy_next = vy + ay_net * dt_val
    return np.array([px_next, py_next, vx_next, vy_next])

def calculate_jacobian_F(x_eval, u_cmd, dt_val, k_d_val):
    px, py, vx, vy = x_eval
    F = np.eye(4)
    s = np.sqrt(vx**2 + vy**2)
    epsilon = 1e-6
    if s < epsilon:
        F[0, 2] = dt_val
        F[1, 3] = dt_val
        F[2, 2] = 1.0
        F[3, 3] = 1.0
    else:
        d_ax_drag_dvx = -k_d_val * (s + vx**2 / s)
        d_ax_drag_dvy = -k_d_val * (vx * vy / s)
        d_ay_drag_dvx = -k_d_val * (vx * vy / s) # 대칭성 사용 가능
        d_ay_drag_dvy = -k_d_val * (s + vy**2 / s)

        F[0, 2] = dt_val + 0.5 * d_ax_drag_dvx * dt_val**2
        F[0, 3] = 0.5 * d_ax_drag_dvy * dt_val**2
        F[1, 2] = 0.5 * d_ay_drag_dvx * dt_val**2
        F[1, 3] = dt_val + 0.5 * d_ay_drag_dvy * dt_val**2
        F[2, 2] = 1 + d_ax_drag_dvx * dt_val
        F[2, 3] = d_ax_drag_dvy * dt_val
        F[3, 2] = d_ay_drag_dvx * dt_val
        F[3, 3] = 1 + d_ay_drag_dvy * dt_val
    return F

# ==============================================================
# 단계 1: 실제 경로 및 제어 입력 시뮬레이션 (원형 궤도 + 속도 보상)
# ==============================================================
print("단계 1: 실제 경로 및 제어 입력 시뮬레이션 중 (원형 궤도 + 속도 보상)...")
true_states = []
control_inputs_log = []
current_true_state = initial_true_state.copy()

for k in range(num_steps):
    px_curr, py_curr, vx_curr, vy_curr = current_true_state

    # 1. 구심 가속도 계산
    dx_to_center = ORBIT_CENTER_X - px_curr
    dy_to_center = ORBIT_CENTER_Y - py_curr
    dist_to_center = np.sqrt(dx_to_center**2 + dy_to_center**2)

    ax_centripetal = 0.0
    ay_centripetal = 0.0
    if dist_to_center > 1e-6:
        norm_x_to_center = dx_to_center / dist_to_center
        norm_y_to_center = dy_to_center / dist_to_center
        ax_centripetal = ACCEL_C_MAG_CMD * norm_x_to_center # 고정 크기 구심 가속도
        ay_centripetal = ACCEL_C_MAG_CMD * norm_y_to_center

    # 2. 접선 방향 가속도 계산 (속도 보상)
    current_speed = np.sqrt(vx_curr**2 + vy_curr**2)
    speed_error = V_TARGET - current_speed
    
    ax_tangential = 0.0
    ay_tangential = 0.0
    if current_speed > 1e-6: # 속도가 0이 아닐 때만 방향을 가짐
        norm_vx = vx_curr / current_speed
        norm_vy = vy_curr / current_speed
        accel_tangential_scalar = KP_SPEED * speed_error
        ax_tangential = accel_tangential_scalar * norm_vx
        ay_tangential = accel_tangential_scalar * norm_vy
    elif V_TARGET > 1e-6: # 정지 상태이고 목표 속도가 있다면 초기 추력 (선택적)
        # 초기 상태에서 vy가 V_TARGET이므로, 정지 상태에서 이 조건이 잘 만족되려면
        # 초기 접선 방향을 알아야 함. 여기서는 초기 속도가 V_TARGET으로 주어져서 큰 문제 없음.
        pass

    # 3. 총 명령 가속도 (구심 + 접선)
    u_k_ax = ax_centripetal + ax_tangential
    u_k_ay = ay_centripetal + ay_tangential
    u_k = np.array([u_k_ax, u_k_ay])

    # 비선형 모델(state_transition_function_f)을 사용하여 다음 실제 상태 결정
    current_true_state_deterministic = state_transition_function_f(current_true_state, u_k, dt, K_DRAG)
    process_noise_vector = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = current_true_state_deterministic + process_noise_vector

    true_states.append(current_true_state.copy())
    control_inputs_log.append(u_k.copy())

print(f"단계 1 완료.")

# ==============================================================
# 단계 2: EKF 적용 (측정값 실시간 생성)
# ==============================================================
print("단계 2: EKF 적용 및 측정값 실시간 생성 중...")
estimated_states = []
measurements_for_viz = []

x_est = x_est_initial.copy()
P = P_initial.copy()

for k in range(num_steps):
    true_state_k = true_states[k]
    u_k = control_inputs_log[k]

    # EKF 예측 단계
    x_pred = state_transition_function_f(x_est, u_k, dt, K_DRAG)
    F_k = calculate_jacobian_F(x_est, u_k, dt, K_DRAG)
    P_pred = F_k @ P @ F_k.T + Q

    # 측정값 생성 (이 코드에서는 위치 x,y를 직접 측정한다고 가정)
    measurement_noise = np.random.multivariate_normal(np.zeros(2), R)
    measurement_k = H @ true_state_k + measurement_noise # H는 위치 측정 행렬
    measurements_for_viz.append(measurement_k.copy())

    # EKF 업데이트 단계
    # H_jac_k는 EKF에서 비선형 측정함수 h를 사용할 때 필요하지만,
    # 여기서는 측정함수 h(x) = Hx (선형) 이므로, H_jac_k = H.
    y = measurement_k - H @ x_pred # Hx_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    
    # 공분산 업데이트 (Joseph form 권장)
    I_KH = np.eye(4) - K @ H
    P = I_KH @ P_pred @ I_KH.T + K @ R @ K.T
    # P = (np.eye(4) - K @ H) @ P_pred # 간단한 형태

    estimated_states.append(x_est.copy())

print("단계 2 완료.")


# ==============================================================
# 단계 3: 시각화 (제어 관련 텍스트 업데이트)
# ==============================================================
print("단계 3: 시각화 시작...")

running = True
frame_index = 0
font = pygame.font.Font(None, 28)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))
    pygame.draw.circle(screen, (200, 200, 200), (int(ORBIT_CENTER_X), int(ORBIT_CENTER_Y)), 5)
    pygame.draw.circle(screen, (220, 220, 220), (int(ORBIT_CENTER_X), int(ORBIT_CENTER_Y)), int(ORBIT_RADIUS), 1)

    if frame_index < num_steps:
        true_state_k_viz = true_states[frame_index]
        # measurement_k_viz = measurements_for_viz[frame_index] # 필요시 사용
        estimated_state_k_viz = estimated_states[frame_index]
        current_u_k_viz = control_inputs_log[frame_index]


        true_x = int(np.clip(true_state_k_viz[0], 0, WIDTH))
        true_y = int(np.clip(true_state_k_viz[1], 0, HEIGHT))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, true_y), 8)

        est_x = int(np.clip(estimated_state_k_viz[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k_viz[1], 0, HEIGHT))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, est_y), 8)
        
        if frame_index > 0:
            prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
            prev_true_y = int(np.clip(true_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (0, 0, 200), (prev_true_x, prev_true_y), (true_x, true_y), 1)
            
            prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
            prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (200, 0, 0), (prev_est_x, prev_est_y), (est_x, est_y), 1)

        info_text_lines = [
            f"스텝: {frame_index+1}/{num_steps} (EKF, 원형CMD, 속도보상)",
            f"K_DRAG: {K_DRAG:.3f}, KP_SPEED: {KP_SPEED:.2f}",
            f"Target Vel: {V_TARGET:.2f}, True Speed: {np.sqrt(true_state_k_viz[2]**2+true_state_k_viz[3]**2):.2f}",
            f"CMD Ax: {current_u_k_viz[0]:.2f}, Ay: {current_u_k_viz[1]:.2f}",
        ]
        for i, line in enumerate(info_text_lines):
            text_surface = font.render(line, True, (0,0,0))
            screen.blit(text_surface, (10, 10 + i*22))

        frame_index += 1
    else:
        font_large = pygame.font.Font(None, 40)
        end_text_surface = font_large.render("EKF (원형 CMD, 속도 보상) 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = end_text_surface.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(end_text_surface, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
단계 1: 실제 경로 및 제어 입력 시뮬레이션 중 (원형 궤도 + 속도 보상)...
단계 1 완료.
단계 2: EKF 적용 및 측정값 실시간 생성 중...
단계 2 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [2]:
import pygame
import numpy as np

# --- Pygame 초기화 ---
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D EKF (거리 측정, 원형 CMD, 속도 보상)") # 창 제목 변경
clock = pygame.time.Clock()

# --- 시뮬레이션 파라미터 ---
dt = 1.0
accel_noise_std = 0.02 # 제어 효과를 보기 위해 약간 줄임
measurement_noise_std = 10.0
num_steps = 600 # 궤도 관찰 시간 증가
K_DRAG = 0.001 # 공기 저항 계수 (적절히 조절)

# --- 원형 궤도 파라미터 ---
ORBIT_CENTER_X = WIDTH / 2.0
ORBIT_CENTER_Y = HEIGHT / 2.0
ORBIT_RADIUS = min(WIDTH, HEIGHT) / 3.0
TARGET_ANGULAR_VELOCITY = 0.04 # 라디안/스텝
V_TARGET = TARGET_ANGULAR_VELOCITY * ORBIT_RADIUS # 목표 접선 속도

# --- 속도 제어 파라미터 ---
KP_SPEED = 0.1  # 속도 오차에 대한 비례 게인 (튜닝 필요)

# --- 센서 위치 정의 (월드 좌표) ---
SENSOR_POS = np.array([WIDTH * 0.8, HEIGHT * 0.8]) # 궤도와 겹치지 않게 센서 위치 조정

# --- 시스템 상태 정의 ---
Q_diag = np.array([
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2
])
Q = np.diag(Q_diag)
R = np.array([[measurement_noise_std**2]])

# --- 초기 실제 상태 (원형 궤도 시작점) ---
initial_px = ORBIT_CENTER_X + ORBIT_RADIUS
initial_py = ORBIT_CENTER_Y
initial_vx = 0.0
initial_vy = V_TARGET # 목표 접선 속도로 시작
initial_true_state = np.array([initial_px, initial_py, initial_vx, initial_vy])

# --- EKF 초기 추정값 ---
x_est_initial = np.array([initial_px + 20, initial_py - 20, 0.0, 0.0]) # 초기 추정 오차
P_initial = np.eye(4) * 100.0

# --- EKF 관련 함수 정의 (이전과 동일) ---
def state_transition_function_f(x_prev, u_cmd, dt_val, k_d_val): # 파라미터 이름 일관성
    px, py, vx, vy = x_prev
    ax_cmd, ay_cmd = u_cmd
    speed = np.sqrt(vx**2 + vy**2)
    ax_drag, ay_drag = 0, 0
    if speed > 1e-6:
        ax_drag = -k_d_val * vx * speed
        ay_drag = -k_d_val * vy * speed
    ax_net = ax_cmd + ax_drag
    ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt_val + 0.5 * ax_net * dt_val**2
    py_next = py + vy * dt_val + 0.5 * ay_net * dt_val**2
    vx_next = vx + ax_net * dt_val
    vy_next = vy + ay_net * dt_val
    return np.array([px_next, py_next, vx_next, vy_next])

def calculate_jacobian_F(x_eval, u_cmd, dt_val, k_d_val): # 파라미터 이름 일관성
    px, py, vx, vy = x_eval
    F = np.eye(4)
    s = np.sqrt(vx**2 + vy**2)
    epsilon = 1e-6
    if s < epsilon:
        F[0, 2] = dt_val; F[1, 3] = dt_val
        F[2, 2] = 1.0; F[3, 3] = 1.0
    else:
        d_ax_drag_dvx = -k_d_val * (s + vx**2 / s)
        d_ax_drag_dvy = -k_d_val * (vx * vy / s)
        d_ay_drag_dvx = d_ax_drag_dvy
        d_ay_drag_dvy = -k_d_val * (s + vy**2 / s)
        F[0, 2] = dt_val + 0.5 * d_ax_drag_dvx * dt_val**2
        F[0, 3] = 0.5 * d_ax_drag_dvy * dt_val**2
        F[1, 2] = 0.5 * d_ay_drag_dvx * dt_val**2
        F[1, 3] = dt_val + 0.5 * d_ay_drag_dvy * dt_val**2
        F[2, 2] = 1 + d_ax_drag_dvx * dt_val
        F[2, 3] = d_ax_drag_dvy * dt_val
        F[3, 2] = d_ay_drag_dvx * dt_val
        F[3, 3] = 1 + d_ay_drag_dvy * dt_val
    return F

def h_measurement_function(x_state, sensor_pos_param):
    px, py = x_state[0], x_state[1]
    sx, sy = sensor_pos_param[0], sensor_pos_param[1]
    dist = np.sqrt((px - sx)**2 + (py - sy)**2)
    return dist

def calculate_jacobian_Hj(x_eval, sensor_pos_param):
    px, py = x_eval[0], x_eval[1]
    sx, sy = sensor_pos_param[0], sensor_pos_param[1]
    dp_x = px - sx
    dp_y = py - sy
    dist_sq = dp_x**2 + dp_y**2
    epsilon = 1e-6
    if dist_sq < epsilon**2:
        return np.array([[0.0, 0.0, 0.0, 0.0]])
    dist = np.sqrt(dist_sq)
    H_j = np.zeros((1, 4))
    H_j[0, 0] = dp_x / dist
    H_j[0, 1] = dp_y / dist
    return H_j

# ==============================================================
# 단계 1: 실제 경로 및 제어 입력 시뮬레이션
# ==============================================================
print("단계 1: 실제 경로 및 제어 입력 시뮬레이션 중...")
true_states = []
control_inputs_log = [] # 이름 변경 (control_inputs는 파이썬 내장 함수와 혼동 가능성)
current_true_state = initial_true_state.copy()

# 루프 내에서 사용할 고정된 구심 가속도 크기 (반지름이 크게 변하지 않는다고 가정)
ACCEL_C_MAG_CMD = TARGET_ANGULAR_VELOCITY**2 * ORBIT_RADIUS

for k in range(num_steps):
    px_curr, py_curr, vx_curr, vy_curr = current_true_state

    # 1. 구심 가속도 계산
    dx_to_center = ORBIT_CENTER_X - px_curr
    dy_to_center = ORBIT_CENTER_Y - py_curr
    dist_to_center = np.sqrt(dx_to_center**2 + dy_to_center**2)

    ax_centripetal = 0.0
    ay_centripetal = 0.0
    if dist_to_center > 1e-6:
        norm_x_to_center = dx_to_center / dist_to_center
        norm_y_to_center = dy_to_center / dist_to_center
        ax_centripetal = ACCEL_C_MAG_CMD * norm_x_to_center # 고정 크기 사용
        ay_centripetal = ACCEL_C_MAG_CMD * norm_y_to_center

    # 2. 접선 방향 가속도 계산 (속도 보상)
    current_speed = np.sqrt(vx_curr**2 + vy_curr**2)
    speed_error = V_TARGET - current_speed
    
    ax_tangential = 0.0
    ay_tangential = 0.0
    if current_speed > 1e-6:
        norm_vx = vx_curr / current_speed
        norm_vy = vy_curr / current_speed
        accel_tangential_scalar = KP_SPEED * speed_error
        ax_tangential = accel_tangential_scalar * norm_vx
        ay_tangential = accel_tangential_scalar * norm_vy
    elif V_TARGET > 1e-6 : # 정지상태이고 목표속도가 있다면 초기 추력 (선택적)
        # 초기 위치에서 접선방향은 (0,1) 이었음 (vy_initial = V_TARGET)
        # 이 경우, 초기 접선 방향으로 약한 추력을 줄 수 있음.
        # 여기서는 초기 속도가 V_TARGET이므로 크게 문제되지 않을 수 있음.
        # 만약 초기 속도가 0이라면, 이 로직이 필요할 수 있음.
        pass


    # 3. 총 명령 가속도
    u_k_ax = ax_centripetal + ax_tangential
    u_k_ay = ay_centripetal + ay_tangential
    u_k = np.array([u_k_ax, u_k_ay])

    current_true_state_deterministic = state_transition_function_f(current_true_state, u_k, dt, K_DRAG)
    process_noise_vector = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = current_true_state_deterministic + process_noise_vector
    true_states.append(current_true_state.copy())
    control_inputs_log.append(u_k.copy())
print("단계 1 완료.")

# ==============================================================
# 단계 2: EKF 적용 (측정값 실시간 생성)
# ==============================================================
print("단계 2: EKF 적용 및 측정값 실시간 생성 중...")
estimated_states = []
measurements_for_viz = []
x_est = x_est_initial.copy()
P = P_initial.copy()

for k in range(num_steps):
    true_state_k = true_states[k]
    u_k = control_inputs_log[k] # 단계 1에서 계산/저장된 제어 입력 사용

    x_pred = state_transition_function_f(x_est, u_k, dt, K_DRAG)
    F_k = calculate_jacobian_F(x_est, u_k, dt, K_DRAG)
    P_pred = F_k @ P @ F_k.T + Q

    true_dist = h_measurement_function(true_state_k, SENSOR_POS)
    measurement_noise_scalar = np.random.normal(0, measurement_noise_std)
    measurement_k = true_dist + measurement_noise_scalar
    measurements_for_viz.append(measurement_k)

    H_j_k = calculate_jacobian_Hj(x_pred, SENSOR_POS)
    z_pred = h_measurement_function(x_pred, SENSOR_POS)
    y = measurement_k - z_pred
    
    S_val_matrix = H_j_k @ P_pred @ H_j_k.T + R # 1x1 행렬
    S_val = S_val_matrix.item()
    
    if np.abs(S_val) < 1e-9:
        K_gain = np.zeros((4,1))
    else:
        K_gain = (P_pred @ H_j_k.T) / S_val

    x_est = x_pred + K_gain.flatten() * y
    I_KH = np.eye(4) - K_gain @ H_j_k
    P = I_KH @ P_pred @ I_KH.T + K_gain @ R @ K_gain.T # Joseph form
    estimated_states.append(x_est.copy())
print("단계 2 완료.")

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0
font = pygame.font.Font(None, 28) # 폰트 크기 약간 조절

sensor_draw_x = int(SENSOR_POS[0])
sensor_draw_y = int(SENSOR_POS[1])

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    screen.fill((255, 255, 255))
    pygame.draw.circle(screen, (200,200,200), (int(ORBIT_CENTER_X), int(ORBIT_CENTER_Y)), 5) # 궤도 중심
    pygame.draw.circle(screen, (220,220,220), (int(ORBIT_CENTER_X), int(ORBIT_CENTER_Y)), int(ORBIT_RADIUS),1) # 이상적 궤도


    if frame_index < num_steps:
        true_state_k_viz = true_states[frame_index]
        measured_dist_k_viz = measurements_for_viz[frame_index]
        estimated_state_k_viz = estimated_states[frame_index]
        current_u_k_viz = control_inputs_log[frame_index]

        true_x = int(np.clip(true_state_k_viz[0], 0, WIDTH))
        true_y = int(np.clip(true_state_k_viz[1], 0, HEIGHT))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, true_y), 8)

        est_x = int(np.clip(estimated_state_k_viz[0], 0, WIDTH))
        est_y = int(np.clip(estimated_state_k_viz[1], 0, HEIGHT))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, est_y), 8)
        
        pygame.draw.rect(screen, (0, 255, 0), (sensor_draw_x - 5, sensor_draw_y - 5, 10, 10))
        draw_radius = max(0, int(measured_dist_k_viz))
        if draw_radius * 2 > max(WIDTH, HEIGHT) * 2 :
             draw_radius = min(WIDTH, HEIGHT) // 2
        pygame.draw.circle(screen, (180, 180, 180), (sensor_draw_x, sensor_draw_y), draw_radius, 1)

        if frame_index > 0:
            prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
            prev_true_y = int(np.clip(true_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (0, 0, 200), (prev_true_x, prev_true_y), (true_x, true_y), 1)
            
            prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
            prev_est_y = int(np.clip(estimated_states[frame_index-1][1], 0, HEIGHT))
            pygame.draw.line(screen, (200, 0, 0), (prev_est_x, prev_est_y), (est_x, est_y), 1)

        info_text_lines = [
            f"스텝: {frame_index+1}/{num_steps} (EKF, 거리, 원형CMD, 속도보상)",
            f"K_DRAG: {K_DRAG:.3f}, KP_SPEED: {KP_SPEED:.2f}, Target Vel: {V_TARGET:.2f}",
            f"CMD Ax: {current_u_k_viz[0]:.2f}, Ay: {current_u_k_viz[1]:.2f}",
            f"True Speed: {np.sqrt(true_state_k_viz[2]**2 + true_state_k_viz[3]**2):.2f}",
            f"측정 거리: {measured_dist_k_viz:.2f}"
        ]
        for i, line in enumerate(info_text_lines):
            text_surface = font.render(line, True, (0,0,0))
            screen.blit(text_surface, (10, 10 + i*22)) # 줄 간격 조절

        frame_index += 1
    else:
        font_large = pygame.font.Font(None, 40) # 완료 메시지 폰트 크기
        text = font_large.render("EKF (원형 CMD, 속도 보상) 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 실제 경로 및 제어 입력 시뮬레이션 중...
단계 1 완료.
단계 2: EKF 적용 및 측정값 실시간 생성 중...
단계 2 완료.
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.
